# Serie 7
## Esercizio 1

### Problema
Il problema di questo esercizio sta nell'accesso concorrente ad una Map condivisa. La motivazione è la mutabilità dell'oggetto.  
Alla rimozione di un elemento il cambiamento non viene propagato a tutti i thread, e si può quindi verificare una Concurrent Modification Exception.  
  
Per ovviare al problema possiamo utilizzare dei blocchi synchronized, oppure sfruttare delle ConcurrentHashMap

### Risoluzione

#### Versione a - Uso dei blocchi synchronized
Utilizzando dei blocchi `synchronized` possiamo evitare le Concurrent Modification Exception:
```java
if (counter == 0) {
    synchronized(sharedMap) {
        if (sharedMap.containsKey(key)
                && sharedMap.get(key).equals(int1)) {
            sharedMap.remove(key);
            log("{" + key + "} remove 1");
        }
    }
}
```

#### Versione b - Uso di ConcurrentHashMap
Utilizzando invece una ConcurrentHashMap e sostituendo i metodi `containsKey` con il rispettivo `get(key) != null` possiamo rendere l'accesso e la modifica della HashMap thread-safe.
```java
private final static Map<String, Integer> sharedMap = new ConcurrentHashMap<String, Integer>();
// ...
while (--cnt > 0) {
    final String key = getClass().getSimpleName()
            + random.nextInt(S7Esercizio1.NUM_WORKERS);
    updateCounter(random.nextBoolean());

    if (counter == 0) {
            if (sharedMap.get(key) != null
                    && sharedMap.get(key).equals(int1)) {
                sharedMap.remove(key);
                log("{" + key + "} remove 1");
            }
    }
    // ...
}
```

## Esercizio 2
### Problema
Il problema si verifica durante l'accesso multi-thread all'iteratore di `S7Esercizio2.sharedPhase` in quanto l'oggetto è *mutable*. Può quindi capitare che durante l'accesso concorrente la collection venga iterata su valori non più esistenti: ad esempio`iterator.hasNext()` ritorna true, avviene un context-switch, la collection viene modificata (e.g: un elemento viene rimosso), si rientra nel contesto dell'iteratore e si chiama `iterattor.next()` che lancia un eccezione in quanto `iterator.hasNext()` sarebbe false dopo la modifica.

## Risoluzione
### Utilizzo di *synchronized blocks*
Il problema può essere facilmente risolto utilizzando un blocco `synchronized` nella fase di iterazione e di aggiunta di elementi. In questo modo possiamo assicurarci che non avvengano context-switch fra `iterator.hasNext()` e `iterator.next()`.  
Questa soluzione, seppur funzionante, risulta però inefficiente.

```java
// Synchronized sull'iterazione
synchronized(S7Esercizio2.sharedPhrase) {
    final Iterator<String> iterator = S7Esercizio2.sharedPhrase.iterator();
    while (iterator.hasNext()) {
        sb.append(iterator.next());
        sb.append(" ");
    }
}
```

```java
// Synchronized sull'aggiunta degli elementi
for (int i = 0; i < 10; i++) {
    synchronized (S7Esercizio2.sharedPhrase) {
        S7Esercizio2.sharedPhrase.add(getWord());
    }
    try {
        Thread.sleep(1000);
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}
```

### Utilizzo di Synchronized Collections
Sfruttando il metodo delle *synchronized collections* possiamo eliminare il blocco `synchronized` nell'aggiunta degli elementi.  

Modifichiamo la dichiarazione di sharePhrase come segue:
```java
S7Esercizio2.sharedPhrase = Collections.synchronizedList(list);
```
  
e modifichiamo l'iterazione nel seguente modo:

```java
synchronized (S7Esercizio2.sharedPhrase) {
    for (final Iterator iterator = S7Esercizio2.sharedPhrase.iterator(); iterator.hasNext(); ) {
        final String string = (String) iterator.next();
        sb.append(string);
        sb.append(" ");
    }
}
```

### Utilizzo di Concurrent Collections
Sfruttando le concurrent collections possiamo rendere più efficiente il nostro programma, mantenendo la thread safety necessaria.  
Per farlo utilizziamo una `CopyOnWriteArrayList` nel modo seguente:  
```java
static volatile List<String> sharedPhrase;
// ...
S7Esercizio2.sharedPhrase = new CopyOnWriteArrayList<>(list);
```

aggiungendo gli elementi senza blocchi synchronized ed iterando normalmente con un *for each*:  
```java
for (String s : S7Esercizio2.sharedPhrase) {
    sb.append(s);
    sb.append(" ");
}
```